In [1]:
import pickle
import pandas as pd
import numpy as np

# create dataset for best picture

In [27]:
oscar_df = pd.read_csv("data/the_oscar_award.csv")
oscar_df = oscar_df[oscar_df['year_ceremony'] >= 1937]
oscar_df['category'] = oscar_df['category'].str.lower() 
bp_mask = oscar_df['category'].str.contains('best|outstanding (motion)? picture')

c:\Users\Nick\Anaconda3\envs\NN\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


## Feature Engineering
### Add Nominations and win count

In [28]:
#create all masks
actor_mask = oscar_df['category'].str.contains('actor( in a leading role)?$')
actress_mask = oscar_df['category'].str.contains('actress( in a leading role)?$')
director_mask = oscar_df['category'].str.contains('directing')
writer_mask = oscar_df['category'].str.contains('writing.*screenplay')
sup_actor_mask = oscar_df['category'].str.contains('actor in a supporting role')
sup_actress_mask = oscar_df['category'].str.contains('actress in a supporting role')

mask_dct = {
    'actor': actor_mask,
    'actress': actress_mask,
    'director': director_mask,
    'writer': writer_mask,
    'sup_actor': sup_actor_mask,
    'sup_actress': sup_actress_mask,
    'best_picture': bp_mask,
}

for k,v in mask_dct.items():
    oscar_df['category'] = np.where(v,k,oscar_df['category'])



big5_mask = actor_mask | actress_mask | bp_mask | director_mask | writer_mask
atl_mask = big5_mask | sup_actor_mask | sup_actress_mask

In [29]:
noms = oscar_df.groupby(['year_film', 'film']).size().rename('noms')
wins = oscar_df[oscar_df['winner']==True & ~bp_mask].groupby(['year_film', 'film'])['category'].nunique().rename('wins')

big5_noms = oscar_df[big5_mask].groupby(['year_film', 'film'])['category'].nunique().rename('big5_noms')
big5_wins = oscar_df[~bp_mask & big5_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('big5_wins')

atl_noms = oscar_df[atl_mask].groupby(['year_film', 'film'])['category'].nunique().rename('atl_noms')
atl_wins = oscar_df[~bp_mask & atl_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('atl_wins')

for ser in (noms,wins,big5_noms,big5_wins,atl_noms,atl_wins):
    oscar_df = pd.merge(oscar_df,ser,left_on=['year_film', 'film'],right_index=True,how='left')

oscar_df[['wins','big5_wins','atl_wins']] = oscar_df[['wins','big5_wins','atl_wins']].fillna(0)


In [35]:
noms_df = (~oscar_df[atl_mask].astype({'winner':int}).pivot_table(index=['year_ceremony','film'],columns='category',values='winner').isna())
wins_df = (~oscar_df[atl_mask&(oscar_df['winner'])].astype({'winner':int}).pivot_table(index=['year_ceremony','film'],columns='category',values='wins').isna())

#remove best_film from both datasets
noms_df = noms_df.drop('best_picture',axis=1)
wins_df = wins_df.drop('best_picture',axis=1)

In [37]:
oscar_df = pd.merge(oscar_df,noms_df,left_on=['year_ceremony','film'],right_index=True,how='left',suffixes=('','_noms'))
oscar_df = pd.merge(oscar_df,wins_df,left_on=['year_ceremony','film'],right_index=True,how='left',suffixes=('_noms','_wins'))


In [45]:
oscar_df.isna().sum()
oscar_df[['atl_noms','big5_noms']] = oscar_df[['atl_noms','big5_noms']].fillna(0)
noms_wins_cols = oscar_df.columns[oscar_df.columns.str.contains('_wins|_noms$')]
oscar_df[noms_wins_cols] = oscar_df[noms_wins_cols].fillna(False)

## Filter Best Picture, remove data leakage, split data

In [82]:
model_df = oscar_df[oscar_df['category']=='best_picture'].set_index(['year_ceremony','film']).drop('category',axis=1)
model_df.drop(['year_film','ceremony','name'],axis=1,inplace=True)
model_df = model_df.astype(int)


#remove data leakage
model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']] = model_df[model_df['winner']==1][['wins','big5_wins','atl_wins']]

c:\Users\Nick\Anaconda3\envs\NN\lib\site-packages\pandas\core\frame.py:3062: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [90]:
test_mask = model_df.index.get_level_values(0) >= 2007

train_df = model_df[~test_mask]
test_df = model_df[test_mask]

# Train Model

In [91]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(train_df.drop('winner',axis=1),train_df['winner'])

test_df['predict'] = model.predict_proba(test_df.drop('winner',axis=1))[:,1]

<ipython-input-91-c5098452f353>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predict'] = model.predict_proba(test_df.drop('winner',axis=1))[:,1]


In [92]:
predictions = test_df.groupby('year_ceremony')['predict'].idxmax()
actual = test_df.groupby('year_ceremony')['winner'].idxmax()

In [95]:
(predictions == actual).sum()/len(predictions)

0.6470588235294118